# Part 1: Imports, data uploads and preparation.

As is customary, let us first call the Python libraries needed here, and upload the needed data and code.

In [ ]:
from model import setup, balance_calcs, dynamic_programming, visuals, performance
import pandas as pd
import numpy as np
import datetime

## Loading model and water balance for historical data

In this tutorial we will compute performance, then compare it for the scenarios defined in Tutorial 3. First let's compute the historical water balance.

In [ ]:
# Preparing the model
reservoir_name = 'Conowingo'
downstream_demand_names = ['Environmental']
direct_demand_names = ['Baltimore', 'Chester', 'Nuclear plant']

# Loading the model!
conowingo = setup.define_reservoir(reservoir_name, downstream_demand_names, direct_demand_names)

# Read flow and demand data. We keep this copy of the data for the simulation of different futures.
flows_default = setup.extract_flows(reservoir=conowingo)
display(flows_default.head())

In [ ]:
# First, make a copy of the flows to initialise the water balance
sop_balance = flows_default.copy()  # Keep flows_default as an untouched copy

# Computing the water balance for our standard operating policy (SOP)
balance_calcs.sop_full(reservoir=conowingo, water_flows=sop_balance)

In [ ]:
# Let us create a time series of hydropower releases
sop_hp_release = np.minimum(sop_balance['Release (m3/s)'].values, np.ones(len(sop_balance))*conowingo.hydropower_plant.max_release)

# We plot hydropower releases vs. total releases. What do we see?
fig = visuals.compare_flow_timeseries(reference=pd.Series(sop_balance['Release (m3/s)']),
                                      alternative=pd.Series(sop_hp_release, index=sop_balance.index),
                                      labels=['Total release', 'Hydropower release'], 
                                      first_date=datetime.date(1990, 1, 1), 
                                      last_date=datetime.date(1991, 1, 1))

**Question 1. What is the extra release? What does that mean for hydropower production under SOP?**
  

# Part 2: Optimizing hydropower production

Now we try to maximise hydropower production. Let us use a dynamic program to achieve this!

Dynamic programming has two phases. 

First, a backward optimization phase where we go backwards to decompose the problem of finding a release decision over many days into a sequence of single-day decisions. This backward phase yields release decisions for discretised states for every day. 

Second, we use this table of release decisions to find the actual sequence of decisions going forward.

## 2.1 - Dynamic programming

Let us start with the first step, after declaring some key parameters.

In [ ]:
# Start date of analysis
first_year = 1989
# Number of discretised storage value
nb_storage_points = 11
# Number of release decisions to investigate at each storage value and time step
nb_decisions = 10

**Question 2. Referring to the code in the `model/dynamic_programming.py` document: what are key steps of the backward optimization algorithm? How long do we run it for? According to you, what would happen if we increased that duration?**

In [ ]:
# First we use the backward function to go backward from the final time step to the initial time step.
tables = dynamic_programming.backward_hp_max(reservoir=conowingo, 
                                             water_flows=flows_default, 
                                             first_year=first_year, 
                                             nb_states=nb_storage_points, 
                                             nb_decisions=nb_decisions)

In [ ]:
# Let us save the tables to have a look (optional)
pd.DataFrame(tables[0]).to_csv('release.csv')
pd.DataFrame(tables[1]).to_csv('value.csv')

**Question 3. Let us open the CSV files in Excel. What do we see?**

**Question 4. Still referring to code in `model/dynamic_programming.py`, do you understand why we need the forward phase below as a separate phase from the backward optimization?**

In [ ]:
# Forward phase
hp_max_balance = dynamic_programming.forward_loop(reservoir=conowingo, 
                                                  water_flows=flows_default, 
                                                  year_beg=first_year, 
                                                  release_table=tables[0])

display(hp_max_balance.head())

## 2.2 - Evaluation of optimal policy

Here we look at what the hydropower-maximising policy does and what its consequences are, compared with SOP.

**Question 5. In practice and based on the figures and metrics below, what are the operational consequences of the policy? Its impact on performance?**

In [ ]:
# Comparing releases
fig = visuals.compare_flow_timeseries(reference=pd.Series(sop_balance['Release (m3/s)']),
                                      alternative=pd.Series(hp_max_balance['Release (m3/s)']),
                                      labels=['SOP', 'Max hydropower'], 
                                      first_date=datetime.date(first_year, 1, 1), 
                                      last_date=datetime.date(first_year+1, 12, 31))

In [ ]:
# Comparing storage
fig = visuals.compare_storage_timeseries(reservoir=conowingo, 
                                         storage_1=pd.Series(sop_balance['Storage (m3)']),
                                         storage_2=pd.Series(hp_max_balance['Storage (m3)']),
                                         labels=['SOP', 'Max hydropower'], 
                                         first_date=datetime.date(first_year, 1, 1), 
                                         last_date=datetime.date(first_year+1, 12, 31))

In [ ]:
# Performance metrics for SOP
metrics_ref = performance.all_metrics(conowingo, sop_balance.loc[datetime.date(first_year, 1, 1):datetime.date(first_year+1, 12, 31), :])
display(metrics_ref)

In [ ]:
# Performance metrics for the hydropower maximizing policy
metrics_hp_max = performance.all_metrics(conowingo, hp_max_balance.loc[datetime.date(first_year, 1, 1):datetime.date(first_year+1, 12, 31), :])
display(metrics_hp_max)

In [ ]:
# Hydropower production under SOP
daily_hp_ref = conowingo.daily_production(sop_balance)
hp_annual_ref = daily_hp_ref.resample('YE').sum()/1000
hp_ref = hp_annual_ref[hp_annual_ref.index.year==first_year].iloc[0] + hp_annual_ref[hp_annual_ref.index.year==first_year+1].iloc[0]
print('With SOP, ' + str(first_year) + '-' + str(first_year+1) + ' total production is ' + "{:.0f}".format(hp_ref) + ' GWh.')

# Maximised hydropower production
hp_maximised_daily = conowingo.daily_production(hp_max_balance)
hp_annual_max = hp_maximised_daily.resample('YE').sum()/1000
hp_maxi = hp_annual_max[hp_annual_max.index.year==first_year].iloc[0] + hp_annual_max[hp_annual_max.index.year==first_year+1].iloc[0]
print('After hydropower maximisation, ' + str(first_year) + '-' + str(first_year+1) + ' total production is ' + "{:.0f}".format(hp_maxi) + ' GWh.')

# Increase
print('This is a ' + "{:.2f}".format(100*(hp_maxi-hp_ref)/hp_ref) + '% increase with optimization.')

# Part 3: Another optimization

Can we mitigate this tradeoff by constraining that the hydropower maximisation should not empty the reservoir in ways that disrupt water supply. The first user to see its water supply cut when water levels get lower is the nuclear plant.


## 3-1. Dynamic programming setup and execution.

In [ ]:
# Success condition: enough water. This corresponds to the demand with the shallowest intake being met (i.e., the Nuclear plant))
# Associated volume
threshold_volume = conowingo.volume_from_height(conowingo.total_lake_depth - conowingo.demand_on_site[2].intake_depth)
print(threshold_volume)

In [ ]:
# First we use the backward function to go backward from the final time step to the initial time step.
tables_2 = dynamic_programming.backward_hp_max(reservoir=conowingo, 
                                             water_flows=flows_default, 
                                             first_year=first_year, 
                                             nb_states=nb_storage_points, 
                                             nb_decisions=nb_decisions,
                                             threshold_volume=threshold_volume)

# Forward phase
hp_constrained_balance = dynamic_programming.forward_loop(reservoir=conowingo, 
                                                  water_flows=flows_default, 
                                                  year_beg=first_year, 
                                                  release_table=tables_2[0])

display(hp_constrained_balance)

## 3.2 - Policy evaluation

**Question 6. Based on the figures and metrics below, what is the impact on addint constraints, both on policy and on performance?**

In [ ]:
# Storage
fig = visuals.compare_storage_timeseries(reservoir=conowingo, 
                                         storage_1=pd.Series(sop_balance['Storage (m3)']),
                                         storage_2=pd.Series(hp_max_balance['Storage (m3)']),
                                         storage_3=pd.Series(hp_constrained_balance['Storage (m3)']),
                                         labels=['SOP', 'Max hydropower', 'With constraints'], 
                                         first_date=datetime.date(first_year, 1, 1), 
                                         last_date=datetime.date(first_year+1, 12, 31))

In [ ]:
# But what happens? Let's zoom in!
fig = visuals.compare_storage_timeseries(reservoir=conowingo, 
                                         storage_1=pd.Series(sop_balance['Storage (m3)']),
                                         storage_2=pd.Series(hp_max_balance['Storage (m3)']),
                                         storage_3=pd.Series(hp_constrained_balance['Storage (m3)']),
                                         labels=['SOP', 'Max hydropower', 'With constraints'], 
                                         first_date=datetime.date(first_year, 5, 1), 
                                         last_date=datetime.date(first_year, 6, 1))

In [ ]:
# Do we get better performance by adding constraints? let's see.
metrics_constrained = performance.all_metrics(conowingo, hp_constrained_balance.loc[datetime.date(first_year, 1, 1):datetime.date(first_year+1, 12, 31), :])
display(metrics_constrained)

In [ ]:
# Maximised hydropower production under constraints
hp_constrained_daily = conowingo.daily_production(hp_constrained_balance)
hp_annual_constrained = hp_constrained_daily.resample('YE').sum()/1000
hp_constrained = hp_annual_constrained[hp_annual_constrained.index.year==first_year].iloc[0] + \
                 hp_annual_constrained[hp_annual_constrained.index.year==first_year+1].iloc[0]
print('After hydropower maximisation, ' + str(first_year) + '-' + str(first_year+1) + ' total production is ' + "{:.0f}".format(hp_constrained) + ' GWh.')

# Increase
print('This is a ' + "{:.2f}".format(100*(hp_constrained-hp_ref)/hp_ref) + '% increase with optimization.')

# Part 4: Impact of reservoir size

We have seen that for a reservoir that is small compared with the inflows and uses (i.e., can be filled and emptied in a few days), improvements on SOP to maximise hydropower production are only marginal. But would that be the case with a much larger reservoir?

## 4.1 - Designing a "larger" reservoir

For this we will actually reduce all inflows and demands, including hydropower (i.e., reduce maximum release and installed capacity). This way the storage will look larger compared with its uses. We divide all by 10.

In [ ]:
# New flows
rescaled_flows = flows_default.copy()
rescaled_flows = rescaled_flows / 10

# New hydropower use
res_2 = conowingo
res_2.hydropower_plant.installed_capacity = conowingo.hydropower_plant.installed_capacity / 10
res_2.hydropower_plant.max_release = conowingo.hydropower_plant.max_release / 10

## 4.2 - Computing policies in this rescaled setting

Let us compute the three policies that we have examined until now: SOP, and policies maximizing hydropower, without or with constraints.

In [ ]:
# Computing the water balance for our standard operating policy (SOP)
sop_rescaled = rescaled_flows.copy()
balance_calcs.sop_full(reservoir=conowingo, water_flows=sop_rescaled)

In [ ]:
# Hydropower maximization, no constraints

# First we use the backward function to go backward from the final time step to the initial time step.
tables = dynamic_programming.backward_hp_max(reservoir=res_2, 
                                             water_flows=rescaled_flows, 
                                             first_year=first_year, 
                                             nb_states=nb_storage_points, 
                                             nb_decisions=nb_decisions)

# Forward phase
hp_max_rescaled = dynamic_programming.forward_loop(reservoir=res_2, 
                                                   water_flows=rescaled_flows, 
                                                   year_beg=first_year, 
                                                   release_table=tables[0])

display(hp_max_rescaled.head())

In [ ]:
# Hydropower maximization, constrained

# First we use the backward function to go backward from the final time step to the initial time step.
tables = dynamic_programming.backward_hp_max(reservoir=res_2, 
                                               water_flows=rescaled_flows, 
                                               first_year=first_year, 
                                               nb_states=nb_storage_points, 
                                               nb_decisions=nb_decisions,
                                               threshold_volume=threshold_volume)

# Forward phase
hp_constrained_rescaled = dynamic_programming.forward_loop(reservoir=res_2, 
                                                           water_flows=rescaled_flows, 
                                                           year_beg=first_year, 
                                                           release_table=tables[0])

display(hp_constrained_rescaled.head())

## 4.3 Evaluation

**Question 7. How does storage size influence the benefits of optimization?**

In [ ]:
fig = visuals.compare_storage_timeseries(reservoir=res_2, 
                                         storage_1=pd.Series(sop_rescaled['Storage (m3)']),
                                         storage_2=pd.Series(hp_max_rescaled['Storage (m3)']),
                                         storage_3=pd.Series(hp_constrained_rescaled['Storage (m3)']),
                                         labels=['SOP release', 'Release for max hydropower', 'With constraints'], 
                                         first_date=datetime.date(first_year, 1, 1), 
                                         last_date=datetime.date(first_year+1, 12, 31))

In [ ]:
m2_sop = performance.all_metrics(res_2, sop_rescaled.loc[datetime.date(first_year, 1, 1):datetime.date(first_year+1, 12, 31), :])
display(m2_sop)

In [ ]:
m2_hpmax = performance.all_metrics(res_2, hp_max_rescaled.loc[datetime.date(first_year, 1, 1):datetime.date(first_year+1, 12, 31), :])
display(m2_hpmax)

In [ ]:
m2_constrained = performance.all_metrics(res_2, hp_constrained_rescaled.loc[datetime.date(first_year, 1, 1):datetime.date(first_year+1, 12, 31), :])
display(m2_constrained)

In [ ]:
# Hydropower production under SOP
daily_hp_ref = res_2.daily_production(sop_rescaled)
hp_annual_ref = daily_hp_ref.resample('YE').sum()/1000
hp_ref = hp_annual_ref[hp_annual_ref.index.year==first_year].iloc[0] + hp_annual_ref[hp_annual_ref.index.year==first_year+1].iloc[0]
print('With SOP, ' + str(first_year) + '-' + str(first_year+1) + ' total production is ' + "{:.0f}".format(hp_ref) + ' GWh.\n')

# Maximised hydropower production
hp_maximised_daily = res_2.daily_production(hp_max_rescaled)
hp_annual_max = hp_maximised_daily.resample('YE').sum()/1000
hp_maxi = hp_annual_max[hp_annual_max.index.year==first_year].iloc[0] + hp_annual_max[hp_annual_max.index.year==first_year+1].iloc[0]
print('After hydropower maximisation, ' + str(first_year) + '-' + str(first_year+1) + ' total production is ' + "{:.0f}".format(hp_maxi) + ' GWh.')
# Increase
print('This is a ' + "{:.2f}".format(100*(hp_maxi-hp_ref)/hp_ref) + '% increase with optimization.\n')

# Maximised hydropower production under constraints
hp_constrained_daily = res_2.daily_production(hp_constrained_rescaled)
hp_annual_constrained = hp_constrained_daily.resample('YE').sum()/1000
hp_constrained = hp_annual_constrained[hp_annual_constrained.index.year==first_year].iloc[0] + \
                 hp_annual_constrained[hp_annual_constrained.index.year==first_year+1].iloc[0]
print('After hydropower maximisation, ' + str(first_year) + '-' + str(first_year+1) + ' total production is ' + "{:.0f}".format(hp_constrained) + ' GWh.')
# Increase
print('This is a ' + "{:.2f}".format(100*(hp_constrained-hp_ref)/hp_ref) + '% increase with optimization.')

Last but not least, you can clean up the CSV files before logging off (especially if working on home machine).